In [ ]:
import pandas as pd
import numpy as np
import re
import os
import nlpaug.augmenter.word as naw
import nltk
from nltk.corpus import stopwords

# ==========================================
# Uncomment these lines on the first run
# ==========================================
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('stopwords') 

class Task2bSmartPreprocessor:
    """
    Smart Preprocessing pipeline for SemEval Task 2b (Disposition Change).
    Features:
    1. Smart Cleaning (Stopword removal with whitelist, strict regex).
    2. Hierarchical Sorting (User -> Group -> Time).
    3. User-Level Augmentation strategy.
    """
    def __init__(self, augment=False):
        """
        :param augment: Boolean. Enable data augmentation (True for Training).
        """
        self.augment = augment
        
        print(">>> [Init] Loading Stopwords with Exceptions...")
        # 1. Load NLTK default stopwords
        stop_words_set = set(stopwords.words('english'))
        
        # 2. Define Whitelist: Words critical for sentiment that MUST NOT be removed
        exceptions = {
            'no', 'not', 'nor', 'neither', 'never', 'none', # Negation
            'but', 'however', 'although', 'except',         # Contrast
            'against', 'cannot', "can't", "don't"           # Other negatives
        }
        
        # 3. Remove exceptions from the stopword list
        self.stop_words = stop_words_set - exceptions
        print(f"    - Preserved {len(exceptions)} emotional keywords (e.g., not, but...)")

        if self.augment:
            print(">>> [Init] Initializing Augmentation Model (WordNet)...")
            self.aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.2)

    def clean_text(self, text, is_words_mode=False):
        """
        Smart Cleaning Logic:
        1. Fix contractions (to protect 'not').
        2. Remove ALL special characters and numbers (keep only A-Z).
        3. Remove stopwords (except whitelist).
        """
        if pd.isna(text): return ""
        text = str(text)

        # --- 1. Fix Contractions FIRST ---
        # We must expand "n't" to "not" before removing punctuation.
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"\s+'\s+t\b", " not", text) # Fix "don ' t"
        text = re.sub(r"\s+'\s+m\b", " am", text)
        text = re.sub(r"\s+'\s+s\b", " is", text)
        text = re.sub(r"\s+'\s+re\b", " are", text)
        text = re.sub(r"\s+'\s+ll\b", " will", text)
        text = re.sub(r"\s+'\s+ve\b", " have", text)
        
        # --- 2. Remove Special Characters & Digits ---
        # Regex: Keep ONLY alphabets (a-z, A-Z) and spaces.
        # This removes '.', ',', '!', '?', '123', etc.
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)

        # --- 3. Normalize Whitespace ---
        text = re.sub(r'\s+', ' ', text).strip()

        # --- 4. Smart Stopword Removal ---
        words = text.split()
        # Filter: Remove word if it is in stop_words (and not in our whitelist)
        filtered_words = [w for w in words if w.lower() not in self.stop_words]
        text = " ".join(filtered_words)

        return text

    def process(self, file_path, output_path):
        print(f"--- Processing File: {file_path} ---")
        
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Input file not found: {file_path}")

        df = pd.read_csv(file_path)
        print(f"Original dataset shape: {df.shape}")
        
        # --- Step 1: Strict Hierarchical Sorting (User -> Group -> Time) ---
        # Crucial for Task 2b which groups texts by user stages.
        if 'timestamp' in df.columns:
            print(">>> [Sorting] Sorting data by User, Group, and Timestamp...")
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            # Sort by User, then Group ID, then Time
            df = df.sort_values(by=['user_id', 'group', 'timestamp'])

        # --- Step 2: Text Cleaning ---
        print(">>> [Cleaning] Applying Smart Cleaning...")
        # Check if 'is_words' column exists
        if 'is_words' in df.columns:
            df['processed_text'] = df.apply(
                lambda row: self.clean_text(row['text'], row['is_words']), axis=1
            )
        else:
            df['processed_text'] = df['text'].apply(lambda x: self.clean_text(x))

        # --- Step 3: Data Augmentation (Training Only) ---
        if self.augment:
            print(">>> [Augmentation] Applying data augmentation...")
            augmented_rows = []
            
            for _, row in df.iterrows():
                original_text = row['processed_text']
                
                # Only augment if the sentence is long enough (> 2 words)
                if len(original_text.split()) > 2:
                    try:
                        # Generate augmented text
                        aug_text = self.aug.augment(original_text)[0]
                        
                        new_row = row.copy()
                        new_row['processed_text'] = aug_text
                        new_row['is_augmented'] = True # Mark as augmented
                        
                        augmented_rows.append(new_row)
                    except:
                        pass # Skip if augmentation fails
            
            # Merge augmented data
            if augmented_rows:
                aug_df = pd.DataFrame(augmented_rows)
                df['is_augmented'] = False
                df = pd.concat([df, aug_df], ignore_index=True)
                print(f"Augmentation complete. Added {len(aug_df)} new samples.")

        # Save to CSV
        df.to_csv(output_path, index=False)
        print(f"Processing finished! File saved to: {output_path}")
        return df

# ==========================================
# Main Execution Block
# ==========================================
if __name__ == "__main__":
    # Logic to locate the file in the parent 'data' directory
    base_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
    
    # Path: ../data/train_release_3sep2025/train_subtask2b_detailed.csv
    input_filename = os.path.join(base_dir, "..", "data", "train_release_3sep2025", "train_subtask2b_detailed.csv")
    
    # Fallback: Check local directory
    if not os.path.exists(input_filename):
        print(f"File not found in parent directory. Checking current directory...")
        input_filename = "train_subtask2b_detailed.csv"

    output_filename = "train_subtask2b_processed_smart.csv"

    # Instantiate and Run
    # Set augment=True to generate Training data
    processor = Task2bSmartPreprocessor(augment=True)
    
    try:
        df_result = processor.process(input_filename, output_filename)
        
        # Verify Results
        print("\n--- Cleaning Verification (User 1 Sample) ---")
        if not df_result.empty:
            sample = df_result.iloc[0]
            print(f"Original: {sample['text'][:60]}...")
            print(f"Cleaned : {sample['processed_text'][:60]}...")
            
    except Exception as e:
        print(f"\n[Error] An error occurred: {e}")

>>> [Init] Initializing Data Augmentation Model (WordNet)...
--- Processing File: /Users/van/Desktop/LLM/2025/Project/SemEval2026-EmoVA/src/../data/train_release_3sep2025/train_subtask2b_detailed.csv ---
Original dataset shape: (2764, 17)
Sorting data by User, Group, and Timestamp...
Cleaning text data...
Applying Data Augmentation (Generating 'Parallel User' samples)...


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package ave

Augmentation complete. Added 0 new samples.
Processing finished! File saved to: train_subtask2b_processed.csv

--- Data Preview (Head of User 1) ---
   user_id            timestamp  \
0        1  2021-06-09 12:41:57   
1        1  2021-06-11 12:01:45   
2        1  2021-06-13 13:15:07   
3        1  2021-06-16 12:03:12   
4        1  2021-06-17 12:38:38   

                                      processed_text  \
0  I feel good . I caught up on some sleep . Work...   
1  I’ve been feeling good for days and days . I r...   
2  I’ve been feeling fine personally . I’ve been ...   
3  I feel great . I’ve had a day off . I’m going ...   
4  I feel great today . I am well rested . I have...   

   disposition_change_valence  
0                    0.666667  
1                    0.666667  
2                    0.666667  
3                    0.666667  
4                    0.666667  


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/van/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package ave